In [ ]:
#-- Install-------------------------------------------------------------------------
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
#-- Import -------------------------------------------------------------------------
from ultralytics import YOLO

import torch

import numpy as np
import pandas as pd

import cv2
from IPython import display
import matplotlib.pyplot as plt

import shutil
import os

import random

In [ ]:
#-- Initialize -------------------------------------------------------------------
output_path = '/kaggle/working/'
input_path = '/kaggle/input/'

ds_path = input_path + 'drone-yolo-detection/Database1/Database1'

train_dir = output_path + 'train/images/'
val_dir = output_path + 'valid/images/'
test_dir = output_path + 'test/images/'

train_lbl_dir = output_path + 'train/labels/'
val_lbl_dir = output_path + 'valid/labels/'
test_lbl_dir = output_path + 'test/labels/'


NUM_EPOCHS = 50

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:' , DEVICE)

data_config_file = 'data.yaml'

CONF_THRESHOLDS = [0.25, 0.5, 0.75, 0.9]
IOU_THRESHOLDS = [0.5, 0.6, 0.7, 0.8, 0.9]
NUMBER_OF_FREEZED_LAYERS = [2, 5, 7, 10, 12, 15]

In [ ]:
#-- Split Dataset to Train, Test, and Validation --------------------------------------------------

def split_dataset(source_dir, train_dir, test_dir, val_dir, train_lbl_dir, test_lbl_dir, val_lbl_dir, split_ratio=(0.7, 0.1, 0.2)):
    
    image_files = [file for file in os.listdir(source_dir) if file.endswith('.JPEG')]
    labels_file = [file for file in os.listdir(source_dir) if file.endswith('.txt')]
    total_images = len(image_files)
    
    print('total_images:' , total_images) 
    print('labels_file:' , len(labels_file))

    # Calculate the number of files for each split
    num_train = int(split_ratio[0] * total_images)
    num_test = int(split_ratio[1] * total_images)
    num_validation = total_images - num_train - num_test
    
    print(f'train_size = {num_train}\nval_size = {num_validation}\ntest_size = {num_test}')    

    # Randomly shuffle the list of files
    random.shuffle(image_files)
    
    # Remove existing train, test, and validation directories
    for directory in [train_dir, test_dir, val_dir, train_lbl_dir, test_lbl_dir, val_lbl_dir]:
        if os.path.exists(directory):
            shutil.rmtree(directory)

    # Create train, test, and validation directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(test_lbl_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)
    
    

    # Copy files to the corresponding directories
    for i, img_file in enumerate(image_files):        
        img_title = img_file.split('.')[0]
        lbl_file = img_title + '.txt'
        
        if lbl_file in labels_file:   
            
            if i < num_train:
                dest_img_dir = train_dir
                dest_lbl_dir = train_lbl_dir
            elif i < num_train + num_test:
                dest_img_dir = test_dir
                dest_lbl_dir = test_lbl_dir
            else:
                dest_img_dir = val_dir
                dest_lbl_dir = val_lbl_dir

            # Copy file (both image and label)
            shutil.copy(os.path.join(source_dir, img_file), os.path.join(dest_img_dir, img_file))
            shutil.copy(os.path.join(source_dir, lbl_file), os.path.join(dest_lbl_dir, lbl_file))
        

In [ ]:
split_dataset(ds_path, train_dir, test_dir, val_dir, train_lbl_dir, test_lbl_dir, val_lbl_dir)

In [ ]:
#-- Create  data.yaml file ------------------------------------------------------------------

file_path = output_path + data_config_file

# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    # Write lines to the file
    file.write('train: ../train/images\n')
    file.write('val: ../valid/images\n')
    file.write('test: ../test/images\n\n')
    file.write('nc: 1\n')
    file.write('names: [\'drone\']')

# Inform the user that writing to the file is complete
print(f"File '{file_path}' has been created and written to.")


In [ ]:
#-- Set test dir as val for evaluting models on test ds ---------------------------
test_data_config_file = 'test_data.yaml'

f1 = open(data_config_file , 'r')
f2 = open(test_data_config_file , 'w')

for line in f1:    
    if 'val:' in line:
        continue
    elif 'test:' in line:
        line = line.replace('test:' , 'val:')
    
    f2.write(line)

f1.close()
f2.close()

In [ ]:
#-- DS Size ------------------------------------------------------------------
train_size = len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])
val_size = len([name for name in os.listdir(val_dir) if os.path.isfile(os.path.join(val_dir, name))])
test_size = len([name for name in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, name))])

print(f'train size:{train_size}\nvalidation size:{val_size}\ntest size:{test_size}')

In [ ]:
#-- Function For Showing Images --------------------------------------------------
def Img_Show(img_file, title=''):  
    
    img = cv2.imread(img_file)

    img = np.asarray(img)
    img_size = img.shape    

    plt.imshow(img, interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])
    plt.title( f'{title}: {img_size}')
    plt.show()   

In [ ]:
# #-- Display Sample Images ---------------------------------------------------
# train_sample_img_file = train_dir + 'img-508_jpg.rf.c5747e8e3af0ece987ca5af6d0819338.jpg'
Img_Show(train_sample_img_file, 'Sample Train Image')

val_sample_img_file = val_dir + 'img-132_jpg.rf.b50598eb1f2ec30f7fa20ec420ef15a9.jpg'
Img_Show(val_sample_img_file, 'Sample Validation Image')

test_sample_img_file = test_dir + 'img-576_jpg.rf.f6cd32a51b0c518b58ff750ecab687d1.jpg'
Img_Show(test_sample_img_file, 'Sample Test Image')

In [ ]:
#-- Train, EValuate, and Save Results -----------------------------------------------------
def Run(model ,number_of_freezed_layers, results_file):
    
    #--  create new folder for each run --
    project_name = 'model_' + str(number_of_freezed_layers) + '_freezed'
    
    #-- Create DF for save results --
    cols_names = ['val_or_test','conf', 'iou', 'map_50_95', 'map_50', 'map_75', 'maps']        
    df_results = pd.DataFrame(columns=cols_names)
    
    #-- Train --
    print('Training Model ---------------------------------------------------------')
    model.train(data = data_config_file,
              epochs = NUM_EPOCHS,
              freeze = number_of_freezed_layers,
              device = DEVICE,
              val = True,
              save = True,
              exist_ok = True,
              plots=True,
              project = project_name,
              name = 'train')
    
    #-- load best model --
    best_model_file = output_path + project_name + '/train/weights/best.pt'
    best_model = YOLO(best_model_file) 
    
    #-- Evaluate Model on Val Data --
    print('Evaluating Model On Val Data --------------------------------------------')
    metrics = best_model.val(data = data_config_file,
                             device = DEVICE,
                             project = project_name,
                             name = 'validation')

    map_50_95 = metrics.box.map
    map_50 = metrics.box.map50
    map_75 = metrics.box.map75
    maps = metrics.box.maps #--a list contains map50-95 of each category --
    
    results = {'val_or_test': 'val',
               'conf':None,
               'iou':None,
               'map_50_95':map_50_95,
               'map_50':map_50,
               'map_75':map_75,
               'maps':maps}

    new_df = pd.DataFrame(results, index=[0])
    df_results = pd.concat([df_results, new_df], ignore_index=True)   

    print(f'map_50_95:{map_50_95}\nmap_50:{map_50}\nmap_75:{map_75}\nmaps:{maps}')

    #-- Evaluate Model Test Data --
    print('Evaluating Model On Test Data --------------------------------------------')
    for conf in CONF_THRESHOLDS:
        for iou in IOU_THRESHOLDS:        
        #-- log --
            print(f'\n\tConf={conf} - IoU={iou} ...................')

            name = 'test/test_' + str(conf) + '_' + str(iou)
            metrics_1 = best_model.val(data = test_data_config_file,
                                       conf = conf,
                                       iou = iou,
                                       device = DEVICE,
                                       project = project_name,
                                       name = name)

            map_50_95 = metrics_1.box.map
            map_50 = metrics_1.box.map50
            map_75 = metrics_1.box.map75
            maps = metrics_1.box.maps #--a list contains map50-95 of each category --
            
            results = {'val_or_test': 'test',
                       'conf':conf,
                       'iou':iou,
                       'map_50_95':map_50_95,
                       'map_50':map_50,
                       'map_75':map_75,
                       'maps':maps}

            #new_df = pd.DataFrame(results, index=[0])
            new_df = pd.DataFrame(results)
            df_results = pd.concat([df_results, new_df], ignore_index=True)   

            print(f'\t\tmap_50_95:{map_50_95}\nmap_50:{map_50}\nmap_75:{map_75}\nmaps:{maps}')
            
    #-- Save DF Rsults --
    df_results.to_csv(output_path + results_file, index=False)            

    
    #-- Run Best Model on Test Images and Save Results --
    print('Running Model on Test Images and Saving Results ----------------------------------')
    for conf in CONF_THRESHOLDS:
        for iou in IOU_THRESHOLDS:        
            #-- log --
            print(f'\n\tConf={conf} - IoU={iou} ......................')

            name = 'predictions_' + str(conf) + '_' + str(iou)

            best_model.predict(source = test_dir,
                               conf = conf,
                               iou = iou,
                               show = False,
                               save= True,
                               project= project_name +'/prediction_results',
                               name=name)   

In [ ]:
#-- Model -------------------------------------------------------------------------------
model = YOLO("yolov8m.pt") 

for n_freezed in NUMBER_OF_FREEZED_LAYERS:
    print(f'n_freezed = {n_freezed} =========================================================')
    results_file = 'model_' + str(n_freezed) + '_results.csv'
    Run(model =  model,  
        number_of_freezed_layers = n_freezed,
        results_file = results_file)